In [1]:
import pandas as pd


In [2]:

Lib1_counts = pd.read_csv("20260217_Lib1_counts_and_scores.csv")
Lib2_counts = pd.read_csv("20260217_Lib2_counts_and_scores.csv")
lib1_scores = pd.read_csv("20260217_TSC2_lib1_scores.csv")
lib2_scores = pd.read_csv("20260217_TSC2_lib2_scores.csv")

print(f"Lib1_counts: {Lib1_counts.shape}")
print(f"Lib2_counts: {Lib2_counts.shape}")
print(f"lib1_scores: {lib1_scores.shape}")
print(f"lib2_scores: {lib2_scores.shape}")

Lib1_counts: (7902, 13)
Lib2_counts: (18050, 13)
lib1_scores: (3951, 8)
lib2_scores: (6017, 9)


In [ ]:
# make a sum column for all the bins for each variant in each replicate
Lib1_counts["sum"] = Lib1_counts[["count_1", "count_2", "count_3", "count_4"]].sum(axis=1)
# split the replicates into two dataframes
rep1 = Lib1_counts[Lib1_counts["replicate"] == 1][["variant", "sum"]].reset_index(drop=True)
rep2 = Lib1_counts[Lib1_counts["replicate"] == 2][["variant", "sum"]].reset_index(drop=True)
# Calculate the frequencies within each replicate  
rep1["frequency"] = rep1["sum"] / rep1["sum"].sum()
rep2["frequency"] = rep2["sum"] / rep2["sum"].sum()
# apply threshold from Matreyek et al., 2018 and generate list of varaints that pass in all replicates
rep1_pass = set(rep1.loc[rep1["frequency"] > threshold, "variant"])
rep2_pass = set(rep2.loc[rep2["frequency"] > threshold, "variant"])

variants_pass = sorted(rep1_pass & rep2_pass)

print(f"Variants passing threshold in both replicates: {len(variants_pass)}")

Variants passing threshold in both replicates: 3943
  rep1 only: 3
  rep2 only: 5


In [12]:
lib1_scores["QC_flag"] = lib1_scores["variant"].isin(variants_pass)

print(f"Passing QC: {lib1_scores['QC_flag'].sum()} / {len(lib1_scores)}")
lib1_scores

Passing QC: 3943 / 3951


,aaChanges,variant,aaChanges_type,abundance_1,abundance_2,abundance_avg,variance,std_dev,QC_flag
0,V156*,ENSP00000219476.3:p.Val709Ter,N,-0.097886,-0.055560,-0.076723,0.000448,0.021163,True
1,P132*,ENSP00000219476.3:p.Pro685Ter,N,-0.041725,0.047948,0.003111,0.002010,0.044836,True
2,P112*,ENSP00000219476.3:p.Pro665Ter,N,0.097905,0.123814,0.110860,0.000168,0.012955,True
3,A30*,ENSP00000219476.3:p.Ala583Ter,N,-0.016799,-0.074520,-0.045660,0.000833,0.028860,True
4,E202*,ENSP00000219476.3:p.Glu755Ter,N,-0.037540,-0.077817,-0.057678,0.000406,0.020138,True
...,...,...,...,...,...,...,...,...,...
3946,E162W,ENSP00000219476.3:p.Glu715Trp,M,0.707929,0.923987,0.815958,0.011670,0.108029,True
3947,L169Q,ENSP00000219476.3:p.Leu722Gln,M,0.795435,0.831680,0.813558,0.000328,0.018123,True
3948,S182M,ENSP00000219476.3:p.Ser735Met,M,0.856474,0.891518,0.873996,0.000307,0.017522,True
3949,S134T,ENSP00000219476.3:p.Ser687Thr,M,0.633401,0.714158,0.673780,0.001630,0.040378,True


In [14]:
#drop the two aaChanges columns that aren't indexed correctly and save the file
lib1_scores = lib1_scores.drop(columns=["aaChanges", "aaChanges_type"])
lib1_scores.to_csv("20260217_TSC2_lib1_scores_QC.csv", index=False)


In [15]:
# make a sum column for all the bins for each variant in each replicate
Lib2_counts["sum"] = Lib2_counts[["count_1", "count_2", "count_3", "count_4"]].sum(axis=1)
# split the replicates into three dataframes
lib2_rep1 = Lib2_counts[Lib2_counts["replicate"] == 1][["variant", "sum"]].reset_index(drop=True)
lib2_rep2 = Lib2_counts[Lib2_counts["replicate"] == 2][["variant", "sum"]].reset_index(drop=True)
lib2_rep3 = Lib2_counts[Lib2_counts["replicate"] == 3][["variant", "sum"]].reset_index(drop=True)
# Calculate the frequencies within each replicate
lib2_rep1["frequency"] = lib2_rep1["sum"] / lib2_rep1["sum"].sum()
lib2_rep2["frequency"] = lib2_rep2["sum"] / lib2_rep2["sum"].sum()
lib2_rep3["frequency"] = lib2_rep3["sum"] / lib2_rep3["sum"].sum()
# apply threshold and generate list of variants that pass in at least 2 of 3 replicates
lib2_rep1_pass = set(lib2_rep1.loc[lib2_rep1["frequency"] > threshold, "variant"])
lib2_rep2_pass = set(lib2_rep2.loc[lib2_rep2["frequency"] > threshold, "variant"])
lib2_rep3_pass = set(lib2_rep3.loc[lib2_rep3["frequency"] > threshold, "variant"])

all_variants = lib2_rep1_pass | lib2_rep2_pass | lib2_rep3_pass
lib2_variants_pass = sorted(
    v for v in all_variants
    if sum([v in lib2_rep1_pass, v in lib2_rep2_pass, v in lib2_rep3_pass]) >= 2
)

print(f"Variants passing threshold in at least 2 of 3 replicates: {len(lib2_variants_pass)}")
print(f"  all 3 replicates: {len(lib2_rep1_pass & lib2_rep2_pass & lib2_rep3_pass)}")
print(f"  exactly 2 replicates: {len(lib2_variants_pass) - len(lib2_rep1_pass & lib2_rep2_pass & lib2_rep3_pass)}")

Variants passing threshold in at least 2 of 3 replicates: 5869
  all 3 replicates: 5751
  exactly 2 replicates: 118


In [16]:
#make the QC flag here too
lib2_scores["QC_flag"] = lib2_scores["variant"].isin(lib2_variants_pass)

print(f"Passing QC: {lib2_scores['QC_flag'].sum()} / {len(lib2_scores)}")
lib2_scores

Passing QC: 5869 / 6017


,aaChanges,variant,aaChanges_type,abundance_1,abundance_2,abundance_3,abundance_avg,variance,std_dev,QC_flag
0,T150-,ENSP00000219476.3:p.Thr1661del,?,0.351778,0.459867,0.440844,0.417497,0.002220,0.047114,True
1,R195-,ENSP00000219476.3:p.Arg1706del,?,-0.075026,-0.099882,0.026900,-0.049336,0.003009,0.054854,True
2,N170-,ENSP00000219476.3:p.Asn1681del,?,0.267801,0.032461,0.180969,0.160410,0.009442,0.097171,True
3,T31*,ENSP00000219476.3:p.Thr1542Ter,N,0.609603,1.155290,0.478297,0.747730,0.085926,0.293132,True
4,D223*,ENSP00000219476.3:p.Asp1734Ter,N,-0.020407,0.247264,0.449404,0.225421,0.037026,0.192421,True
...,...,...,...,...,...,...,...,...,...,...
6012,H109V,ENSP00000219476.3:p.His1620Val,M,0.877959,0.815445,0.716251,0.803218,0.004433,0.066580,True
6013,I161H,ENSP00000219476.3:p.Ile1672His,M,0.709234,0.818513,0.757211,0.761653,0.002000,0.044723,True
6014,E17R,ENSP00000219476.3:p.Glu1528Arg,M,1.101614,0.982971,1.089553,1.058046,0.002842,0.053314,True
6015,L183K,ENSP00000219476.3:p.Leu1694Lys,M,0.792926,0.401362,0.608158,0.600815,0.025581,0.159940,True


In [ ]:
# drop the two aaChanges columns that aren't indexed correctly again and save the file
lib2_scores = lib2_scores.drop(columns=["aaChanges", "aaChanges_type"])
lib2_scores.to_csv("20260217_TSC2_lib2_scores_QC.csv", index=False)
